## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

DATA_PATH = "/content/drive/MyDrive/data/"
SEED = 42

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 17.4 MB/s eta 0:00:00


In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import warnings
import joblib
warnings.filterwarnings('ignore')

In [ ]:
import sklearn,xgboost,lightgbm,catboost,sys,platform
print("numpy_version:",np.__version__)
print("pandas_version:",pd.__version__)
print("sklearn_version:",sklearn.__version__)
print("xgboost_version:",xgboost.__version__)
print("lightgbm_version:",lightgbm.__version__)
print("catboost_version:",catboost.__version__)
print(f"python_version: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print("OS :",platform.platform())
# numpy_version: 1.23.5
# pandas_version: 1.5.3
# sklearn_version: 1.2.2
# xgboost_version: 1.7.6
# lightgbm_version: 4.0.0
# catboost_version: 1.2

numpy_version: 1.23.5
pandas_version: 1.5.3
sklearn_version: 1.2.2
xgboost_version: 1.7.6
lightgbm_version: 4.0.0
catboost_version: 1.2


In [ ]:
reset_seeds(SEED)

## Load Data

In [ ]:
train_df = pd.read_csv(f'{DATA_PATH}train.csv')
test_df = pd.read_csv(f'{DATA_PATH}test.csv')

In [ ]:
building_info = pd.read_csv(f"{DATA_PATH}building_info.csv")

In [ ]:
date = pd.to_datetime(train_df['일시'])
train_df['hour'] = date.dt.hour
train_df['day'] = date.dt.weekday
train_df['month'] = date.dt.month
train_df['week'] = date.dt.weekofyear

In [ ]:
date = pd.to_datetime(test_df['일시'])
test_df['hour'] = date.dt.hour
test_df['day'] = date.dt.weekday
test_df['month'] = date.dt.month
test_df['week'] = date.dt.weekofyear

# 빌딩정보 전처리

In [ ]:
building_info

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,-,-,-
96,97,호텔및리조트,55144.67,25880.00,-,-,-
97,98,호텔및리조트,53578.62,17373.75,-,-,-
98,99,호텔및리조트,53499.00,40636.00,-,-,-


In [ ]:
cols = ['태양광용량(kW)','ESS저장용량(kWh)','PCS용량(kW)']

In [ ]:
for col in cols :
    building_info[col] = building_info[col].apply(lambda x : 0 if x =='-' else float(x))

In [ ]:
building_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    float64
 5   ESS저장용량(kWh)  100 non-null    float64
 6   PCS용량(kW)     100 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 5.6+ KB


# train 데이터 결측치처리

In [ ]:
train_df.info()
# 강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_date_time  204000 non-null  object 
 1   건물번호           204000 non-null  int64  
 2   일시             204000 non-null  object 
 3   기온(C)          204000 non-null  float64
 4   강수량(mm)        43931 non-null   float64
 5   풍속(m/s)        203981 non-null  float64
 6   습도(%)          203991 non-null  float64
 7   일조(hr)         128818 non-null  float64
 8   일사(MJ/m2)      116087 non-null  float64
 9   전력소비량(kWh)     204000 non-null  float64
 10  hour           204000 non-null  int64  
 11  day            204000 non-null  int64  
 12  month          204000 non-null  int64  
 13  week           204000 non-null  int64  
dtypes: float64(7), int64(5), object(2)
memory usage: 21.8+ MB


In [ ]:
test_df.info() # 결측치 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num_date_time  16800 non-null  object 
 1   건물번호           16800 non-null  int64  
 2   일시             16800 non-null  object 
 3   기온(C)          16800 non-null  float64
 4   강수량(mm)        16800 non-null  float64
 5   풍속(m/s)        16800 non-null  float64
 6   습도(%)          16800 non-null  int64  
 7   hour           16800 non-null  int64  
 8   day            16800 non-null  int64  
 9   month          16800 non-null  int64  
 10  week           16800 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 1.4+ MB


In [ ]:
train_ft = train_df.drop(columns=['num_date_time', '일시','일조(hr)', '일사(MJ/m2)'])
test_ft = test_df.drop(columns=['num_date_time', '일시'])

In [ ]:
train_ft['강수량(mm)']= train_ft['강수량(mm)'].fillna(0)

- 풍속, 습도 결측치 위치 확인

In [ ]:
train_ft[train_ft['풍속(m/s)'].isnull()]

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh),hour,day,month,week
16643,9,17.6,0.0,NaN,NaN,2330.40,11,1,6,24
30161,15,35.6,0.0,NaN,NaN,1815.57,17,5,8,31
32127,16,30.4,0.0,NaN,77.0,5062.08,15,2,8,31
51640,26,27.2,3.1,NaN,93.0,5330.40,16,0,6,26
51921,26,26.3,0.0,NaN,84.0,2908.80,9,5,7,27
84418,42,29.8,0.0,NaN,66.0,3038.76,10,6,7,26
100738,50,29.8,0.0,NaN,66.0,3044.70,10,6,7,26
105639,52,33.4,0.0,NaN,72.0,2260.20,15,5,8,31
105640,52,33.5,0.0,NaN,73.0,2274.30,16,5,8,31
176477,87,21.9,0.0,NaN,NaN,519.66,5,3,7,28


In [ ]:
train_ft[(train_ft['건물번호']==9)&(train_ft['month']==6) & (train_ft['hour'] ==11)]['풍속(m/s)'].describe()

count    29.000000
mean      2.941379
std       1.144090
min       1.300000
25%       2.200000
50%       2.600000
75%       3.500000
max       5.900000
Name: 풍속(m/s), dtype: float64

In [ ]:
for i in train_ft[train_ft['풍속(m/s)'].isnull()].index:
    # display(train_ft.iloc[i])
    month = train_ft.iloc[i]['month']
    hour = train_ft.iloc[i]['hour']
    num = train_ft.iloc[i]['건물번호']
    data_mean = train_ft[(train_ft['건물번호']==num)&(train_ft['month']==month) & (train_ft['hour'] ==hour)]['풍속(m/s)'].describe()['mean']
    train_ft.loc[i,'풍속(m/s)'] = data_mean

In [ ]:
train_ft['풍속(m/s)'].isnull().sum()

0

In [ ]:
train_ft[(train_ft['month']==6) & (train_ft['hour'] ==11)]['습도(%)'].describe()

count    2999.000000
mean       66.573525
std        17.162310
min        19.000000
25%        53.000000
50%        66.000000
75%        80.000000
max       100.000000
Name: 습도(%), dtype: float64

In [ ]:
for i in train_ft[train_ft['습도(%)'].isnull()].index:
    # display(train_ft.iloc[i])
    month = train_ft.iloc[i]['month']
    hour = train_ft.iloc[i]['hour']
    num = train_ft.iloc[i]['건물번호']
    data_mean = train_ft[(train_ft['건물번호']==num)&(train_ft['month']==month) & (train_ft['hour'] ==hour)]['습도(%)'].describe()['mean']
    train_ft.loc[i,'습도(%)'] = data_mean


In [ ]:
train_ft['습도(%)'].isnull().sum()

0

# building_info 합해주기

In [ ]:
building_info

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,0.0,0.0,0.0
1,2,건물기타,122233.47,99000.00,0.0,0.0,0.0
2,3,건물기타,171243.00,113950.00,40.0,0.0,0.0
3,4,건물기타,74312.98,34419.62,60.0,0.0,0.0
4,5,건물기타,205884.00,150000.00,0.0,2557.0,1000.0
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,0.0,0.0,0.0
96,97,호텔및리조트,55144.67,25880.00,0.0,0.0,0.0
97,98,호텔및리조트,53578.62,17373.75,0.0,0.0,0.0
98,99,호텔및리조트,53499.00,40636.00,0.0,0.0,0.0


In [ ]:
train_ft = train_ft.merge(building_info,how='left',on='건물번호')
test_ft = test_ft.merge(building_info,how='left',on='건물번호')

# 범주형 인코딩

In [ ]:
conv_dict = dict(zip(list(train_ft['건물유형'].unique()), [0,1,2,3,4,5,6,7,8,9,10,11]))
conv_dict

{'건물기타': 0,
 '공공': 1,
 '대학교': 2,
 '데이터센터': 3,
 '백화점및아울렛': 4,
 '병원': 5,
 '상용': 6,
 '아파트': 7,
 '연구소': 8,
 '지식산업센터': 9,
 '할인마트': 10,
 '호텔및리조트': 11}

In [ ]:
train_ft['건물유형'] = train_ft['건물유형'].map(conv_dict)
test_ft['건물유형'] = test_ft['건물유형'].map(conv_dict)

# target 만들기

In [ ]:
target = train_df['전력소비량(kWh)']
target

0         1085.28
1         1047.36
2          974.88
3          953.76
4          986.40
           ...   
203995     881.04
203996     798.96
203997     825.12
203998     640.08
203999     540.24
Name: 전력소비량(kWh), Length: 204000, dtype: float64

# 공휴일 더하기

In [ ]:
train_ft['holiday'] = train_ft.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
test_ft['holiday'] = train_ft.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)

In [ ]:
train_ft['sin_time'] = np.sin(2*np.pi*train_ft.hour/24)
train_ft['cos_time'] = np.cos(2*np.pi*train_ft.hour/24)
test_ft['sin_time'] = np.sin(2*np.pi*test_ft.hour/24)
test_ft['cos_time'] = np.cos(2*np.pi*test_ft.hour/24)

# 불쾌지수

In [ ]:
train_ft['THI'] = 9/5*train_ft['기온(C)'] - 0.55*(1-train_ft['습도(%)']/100)*(9/5*train_ft['습도(%)']-26)+32
test_ft['THI'] = 9/5*test_ft['기온(C)'] - 0.55*(1-test_ft['습도(%)']/100)*(9/5*test_ft['습도(%)']-26)+32

In [ ]:
train_ft['THI'] = pd.cut(train_ft['THI'], bins = [0, 68, 75, 80, 200], labels = [1,2,3,4])
train_ft['THI'] = train_ft['THI'].astype(int)
test_ft['THI'] = pd.cut(test_ft['THI'], bins = [0, 68, 75, 80, 200], labels = [1,2,3,4])
test_ft['THI'] = test_ft['THI'].astype(int)

In [ ]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

In [ ]:
cdhs = np.array([])
for num in range(1,101,1):
    temp = train_ft[train_ft['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
train_ft['CDH'] = cdhs

In [ ]:
cdhs = np.array([])
for num in range(1,101,1):
    temp = test_ft[test_ft['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
test_ft['CDH'] = cdhs

# 요일 시간별 사용량

In [ ]:
power_mean = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day','month'], aggfunc = np.mean).reset_index()

power_mean = power_mean.drop_duplicates(subset=['건물번호', 'hour', 'day'], keep='first')

train_ft = train_ft.merge(power_mean[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                          on=['건물번호', 'hour', 'day'],
                          how='left',
                          suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'day_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                        on=['건물번호', 'hour', 'day'],
                        how='left',
                        suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_mean'}, inplace=True)


In [ ]:
power_mean = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day','month'], aggfunc = np.mean).reset_index()

power_mean = power_mean.drop_duplicates(subset=['건물번호', 'hour', 'month'], keep='first')

train_ft = train_ft.merge(power_mean[['건물번호', 'hour', 'month', '전력소비량(kWh)']],
                          on=['건물번호', 'hour', 'month'],
                          how='left',
                          suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'month_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', 'month', '전력소비량(kWh)']],
                        on=['건물번호', 'hour', 'month'],
                        how='left',
                        suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'month_hour_mean'}, inplace=True)


In [ ]:
power_mean = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.mean).reset_index()
train_ft = train_ft.merge(power_mean[['건물번호', 'hour', '전력소비량(kWh)']],
                          on=['건물번호', 'hour'],
                          how='left',
                          suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', '전력소비량(kWh)']],
                        on=['건물번호', 'hour'],
                        how='left',
                        suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'hour_mean'}, inplace=True)

In [ ]:
power_std = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.std).reset_index()
train_ft = train_ft.merge(power_std[['건물번호', 'hour', '전력소비량(kWh)']],
                          on=['건물번호', 'hour'],
                          how='left',
                          suffixes=('', '_std'))
train_ft.rename(columns={'전력소비량(kWh)_std': 'hour_std'}, inplace=True)

test_ft = test_ft.merge(power_std[['건물번호', 'hour', '전력소비량(kWh)']],
                        on=['건물번호', 'hour'],
                        how='left',
                        suffixes=('', '_std'))
test_ft.rename(columns={'전력소비량(kWh)': 'hour_std'}, inplace=True)

In [ ]:
power_std = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.std).reset_index()
train_ft = train_ft.merge(power_std[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                          on=['건물번호', 'hour', 'day'],
                          how='left',
                          suffixes=('', '_std'))
train_ft.rename(columns={'전력소비량(kWh)_std': 'day_hour_std'}, inplace=True)

test_ft = test_ft.merge(power_std[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                        on=['건물번호', 'hour', 'day'],
                        how='left',
                        suffixes=('', '_std'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_std'}, inplace=True)

In [ ]:
power_median = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.median).reset_index()
train_ft = train_ft.merge(power_median[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                          on=['건물번호', 'hour', 'day'],
                          how='left',
                          suffixes=('', '_median'))
train_ft.rename(columns={'전력소비량(kWh)_median': 'day_hour_median'}, inplace=True)

test_ft = test_ft.merge(power_median[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                        on=['건물번호', 'hour', 'day'],
                        how='left',
                        suffixes=('', '_median'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_median'}, inplace=True)

In [ ]:
train_ft = train_ft.drop(columns=['holiday', '건물유형', '강수량(mm)', '풍속(m/s)','전력소비량(kWh)'])
test_ft = test_ft.drop(columns=['holiday', '건물유형', '강수량(mm)', '풍속(m/s)'])

In [ ]:
cols = ['건물번호', '기온(C)', '습도(%)', 'hour', 'day', 'month', 'week', '연면적(m2)',
       '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', 'sin_time',
       'cos_time', 'THI', 'CDH', 'day_hour_mean', 'month_hour_mean','hour_mean', 'hour_std','day_hour_std',
       'day_hour_median']

In [ ]:
train_ft.to_csv("tmp.csv",index=False)
test_ft.to_csv("tmp2.csv",index=False)

In [ ]:
train_ft = pd.read_csv("./tmp.csv")
test_ft = pd.read_csv("./tmp2.csv")

In [ ]:
train_ft= train_ft[cols]
test_ft = test_ft[cols]

In [ ]:
train_ft['전력소비량(kWh)'] = train_df['전력소비량(kWh)']

In [ ]:
train_ft['holiday'] = train_ft.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
test_ft['holiday'] = train_ft.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)

In [ ]:
weekday_mean = train_ft[train_ft.holiday==0].pivot_table(values='전력소비량(kWh)', index=train_ft.건물번호, columns='hour', aggfunc='first') # 평일 시간당 소비전력

In [ ]:
weekend_mean = train_ft[train_ft.holiday==1].pivot_table(values='전력소비량(kWh)', index=train_ft.건물번호, columns='hour', aggfunc='first')

In [ ]:
tmp = pd.merge(weekday_mean,weekend_mean,how='left',on = '건물번호')
tmp

hour,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y
건물번호,,,,,,,,,,,,,,,,,,,,,
1,1085.28,1047.36,974.88,953.76,986.40,1087.20,1314.72,1684.80,1976.16,2289.12,...,3210.24,3153.60,3035.52,2726.40,2315.04,2229.60,2069.76,1733.76,1414.08,1191.84
2,1170.36,1146.96,1115.28,1107.00,1069.92,1075.32,1116.36,1254.24,887.40,1035.00,...,2245.68,2316.60,2528.28,2201.04,2028.60,1636.56,1380.96,1109.52,811.44,1268.64
3,926.28,884.52,882.36,867.24,880.92,808.56,786.24,809.64,818.64,882.36,...,1596.96,1791.72,1875.60,1746.00,1794.24,2265.48,2087.28,1052.64,972.72,912.60
4,740.16,671.04,619.20,626.40,626.22,630.36,752.04,829.44,957.60,1093.14,...,1237.50,1226.52,1172.70,1064.88,959.04,875.88,816.12,755.10,708.48,656.28
5,1304.64,1308.24,1302.48,1295.28,1316.88,1398.96,1492.56,1535.04,1548.72,1560.96,...,5785.92,5768.64,5553.36,5284.80,2644.56,1645.92,1538.64,1463.04,1404.00,1416.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1576.68,1558.68,1413.36,1483.92,1438.56,1503.84,1738.80,2073.36,2263.80,2621.04,...,3095.16,3043.80,3042.36,2958.84,2883.48,2780.04,2729.28,2525.88,2098.44,1848.72
97,774.36,553.86,558.90,559.44,559.62,582.48,640.26,903.78,855.18,907.56,...,1486.44,1469.88,1471.50,1340.46,1215.90,1180.98,1275.30,1128.42,975.96,895.14
98,533.52,516.78,509.40,479.70,541.98,712.08,947.88,1004.58,1069.02,1133.64,...,1298.16,1296.72,1273.32,1298.70,1238.22,1139.58,1093.68,979.20,783.00,654.30


In [ ]:
kmeans= KMeans(n_clusters=5,random_state=SEED,n_init='auto')
kmeans.fit(tmp)

KMeans(n_clusters=5, n_init='auto', random_state=42)

In [ ]:
cluster_ = kmeans.predict(tmp)

In [ ]:
tmp['cluster'] = cluster_

In [ ]:
train_ft = train_ft.merge(tmp.cluster,how='left',on='건물번호')
test_ft = test_ft.merge(tmp.cluster,how='left',on='건물번호')

In [ ]:
train_ft = train_ft.drop(columns=['holiday'])
test_ft = test_ft.drop(columns=['holiday'])

In [ ]:
power_mean = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['cluster', 'hour', 'day','month'], aggfunc = np.mean).reset_index()

power_mean = power_mean.drop_duplicates(subset=['cluster', 'hour', 'day'], keep='first')

train_ft = train_ft.merge(power_mean[['cluster', 'hour', 'day', '전력소비량(kWh)']],
                          on=['cluster', 'hour', 'day'],
                          how='left',
                          suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'cluster_day_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['cluster', 'hour', 'day', '전력소비량(kWh)']],
                        on=['cluster', 'hour', 'day'],
                        how='left',
                        suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'cluster_day_hour_mean'}, inplace=True)


In [ ]:
display(train_ft)
display(test_ft)

,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,CDH,day_hour_mean,month_hour_mean,hour_mean,hour_std,day_hour_std,day_hour_median,전력소비량(kWh),cluster,cluster_day_hour_mean
0,1,18.6,42.0,0,2,6,22,110634.00,39570.00,0.0,...,-7.4,1174.656,1189.44,1706.318118,446.882767,517.982222,2094.72,1085.28,0,2005.185000
1,1,18.0,45.0,1,2,6,22,110634.00,39570.00,0.0,...,-15.4,1101.600,1131.12,1622.620235,439.662704,500.769931,2015.52,1047.36,0,1952.292000
2,1,17.7,45.0,2,2,6,22,110634.00,39570.00,0.0,...,-23.7,1025.760,1063.56,1506.971294,412.071906,465.227458,1848.00,974.88,0,1918.248500
3,1,16.7,48.0,3,2,6,22,110634.00,39570.00,0.0,...,-33.0,1015.584,1042.68,1437.365647,391.205981,436.601091,1764.00,953.76,0,1894.579500
4,1,18.4,43.0,4,2,6,22,110634.00,39570.00,0.0,...,-40.6,1041.792,1075.44,1447.321412,381.099697,405.518091,1619.52,986.40,0,1901.786500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,86.0,19,2,8,34,57497.84,40035.23,0.0,...,-19.6,832.752,1168.20,1010.462118,161.399578,163.825489,1017.12,881.04,4,1424.581127
203996,100,22.4,86.0,20,2,8,34,57497.84,40035.23,0.0,...,-20.2,773.040,1026.00,928.125176,137.566008,153.076049,874.32,798.96,4,1353.478945
203997,100,21.3,92.0,21,2,8,34,57497.84,40035.23,0.0,...,-22.3,686.784,934.68,830.032941,128.300189,143.415686,781.20,825.12,4,1228.707309
203998,100,21.0,94.0,22,2,8,34,57497.84,40035.23,0.0,...,-25.1,584.736,815.52,723.100235,112.464079,105.147190,657.12,640.08,4,1073.485988


,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,THI,CDH,day_hour_mean,month_hour_mean,hour_mean,hour_std,day_hour_std,day_hour_median,cluster,cluster_day_hour_mean
0,1,23.5,72,0,3,8,34,110634.00,39570.00,0.0,...,1,-2.5,1135.584,2237.04,1706.318118,446.882767,446.984913,1847.28,0,2011.061500
1,1,23.0,72,1,3,8,34,110634.00,39570.00,0.0,...,1,-5.5,1054.848,2165.64,1622.620235,439.662704,449.091398,1772.40,0,1972.697000
2,1,22.7,75,2,3,8,34,110634.00,39570.00,0.0,...,1,-8.8,975.648,2035.92,1506.971294,412.071906,415.453568,1624.80,0,1943.292000
3,1,22.1,78,3,3,8,34,110634.00,39570.00,0.0,...,1,-12.7,956.736,1954.56,1437.365647,391.205981,378.117772,1557.84,0,1925.145000
4,1,21.8,77,4,3,8,34,110634.00,39570.00,0.0,...,1,-16.9,984.768,1960.56,1447.321412,381.099697,360.348483,1561.68,0,1936.258500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,22.5,84,19,2,8,35,57497.84,40035.23,0.0,...,1,-34.5,832.752,1168.20,1010.462118,161.399578,163.825489,1017.12,4,1424.581127
16796,100,20.7,95,20,2,8,35,57497.84,40035.23,0.0,...,1,-34.4,773.040,1026.00,928.125176,137.566008,153.076049,874.32,4,1353.478945
16797,100,20.2,98,21,2,8,35,57497.84,40035.23,0.0,...,1,-35.3,686.784,934.68,830.032941,128.300189,143.415686,781.20,4,1228.707309
16798,100,20.1,97,22,2,8,35,57497.84,40035.23,0.0,...,1,-36.8,584.736,815.52,723.100235,112.464079,105.147190,657.12,4,1073.485988


In [ ]:
train_ft=train_ft.drop(columns=['전력소비량(kWh)'])

In [ ]:
cv= KFold(n_splits = 5,shuffle=True,random_state = SEED)

In [ ]:
model3 = XGBRegressor(random_state=SEED)

In [ ]:
model2 = LGBMRegressor(random_state=SEED)

In [ ]:
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred)*2)/(np.abs(true) + np.abs(pred))) * 100

In [ ]:
scores = cross_val_score(model3,train_ft,target,cv = cv ,scoring=make_scorer(SMAPE,greater_is_better=False),n_jobs = -1)
print(f"XGB : {-np.mean(scores)}")
# XGB : 5.04456619938086

XGB : 5.04456619938086


In [ ]:
scores = cross_val_score(model2,train_ft,target,cv = cv ,scoring=make_scorer(SMAPE,greater_is_better=False),n_jobs = -1)
print(f"LGBM : {-np.mean(scores)}")
# LGBM : 5.6669996037683275

LGBM : 5.6669996037683275


In [ ]:
cat_col = ['건물번호','cluster','THI']
def lgbm_smape(preds, train_data):
    '''
    Custom Evaluation Function for LGBM
    '''
    # labels = train_data.get_label()
    labels = train_data
    smape_val=SMAPE(labels,preds)
    return 'SMAPE', smape_val, False

In [ ]:
reset_seeds(SEED)
models = []
for i in tqdm(range(42,53)):
  print(f"###################[{i}]SEED################################")
  cat_params = {'learning_rate': 0.14059048492476106,
 'loss_function': 'RMSE',
 'random_state': i,
 'verbose': False,
 'train_dir': 'catboost_1690862972.3436863',
#  'n_estimators': 2000,
 'n_estimators': 10000,
#  'early_stopping_rounds': 11,
 'early_stopping_rounds': 100,
 'objective': 'MAE'}

  lgbm_params = {'random_state': i,
          'n_estimators': 15000,
          # 'objective' : weighted_mse(100),
          'num_leaves': 8,
          'min_child_samples': 12,
          'learning_rate': 0.17010396907527026,
          'colsample_bytree': 0.9605563464803123,
          'reg_alpha': 0.1110993344544235,
          'reg_lambda': 0.7948637803974561,
          "verbose" : -1,"n_jobs" : -1}
  xgb_params = {'random_state' : i,
          'n_estimators': 4682,
          'max_leaves': 101,
          'min_child_weight': 7.581207558322951,
          'learning_rate': 0.08979034933474227,
          'subsample': 0.8905280196300354,
          'colsample_bylevel': 1.0,
          'colsample_bytree': 0.9523645407001878,
          'reg_alpha': 0.006919296411231538,
          'reg_lambda': 0.0998936254543762,"n_jobs" : -1}

  Cat_model = CatBoostRegressor(**cat_params)
  LGBM_model = LGBMRegressor(**lgbm_params)
  XGB_model = XGBRegressor(**xgb_params)

  scores = []
  for tri,vai in tqdm(cv.split(train_ft),total=5):
      # 학습데이터
      x_train = train_ft.iloc[tri]
      y_train = target.iloc[tri]

      # 검증데이터
      x_valid = train_ft.iloc[vai]
      y_valid = target.iloc[vai]

      XGB_model.fit(x_train,y_train)
      LGBM_model.fit(x_train,y_train,eval_metric=lgbm_smape,categorical_feature=cat_col,
                   eval_set=(x_valid,y_valid))
      Cat_model.fit(x_train,y_train)

      pred = XGB_model.predict(x_valid)
      pred2 = LGBM_model.predict(x_valid)
      pred3 = Cat_model.predict(x_valid)
      pred = (pred+pred2+pred3)/3

      score = SMAPE(y_valid,pred)
      print(score)
      scores.append(score)
      models.append(XGB_model)
      models.append(LGBM_model)
      models.append(Cat_model)
  print(np.mean(scores))

  0%|          | 0/11 [00:00<?, ?it/s]

###################[42]SEED################################


  0%|          | 0/5 [00:00<?, ?it/s]

3.6770071822870833
3.6917642945898907
3.715263138413786
3.645920119525603
3.630021387685805
3.6719952245004337
###################[43]SEED################################


  0%|          | 0/5 [00:00<?, ?it/s]

...

3.6777645934470984
3.6894290622270445
3.708582459578506
3.6512306587694825
3.632170256569647
3.671835406118356
###################[51]SEED################################


  0%|          | 0/5 [00:00<?, ?it/s]

3.680575514579672
3.686765007753573
3.700727495072674
3.6351213490884446
3.638631969023049
3.6683642671034833
###################[52]SEED################################


  0%|          | 0/5 [00:00<?, ?it/s]

3.6679477592733343
3.6924452766852425
3.7086857018782995
3.6434188260369766
3.62876683563349
3.6682528799014684


In [ ]:
!mkdir /content/drive/MyDrive/data/73번실험

In [ ]:
for i,model in enumerate(models):
    joblib.dump(model,f"{DATA_PATH}73번실험/11seeds_ensemble[{i}].pkl")

In [ ]:
models = []
for i in tqdm(range(165)):
    loaded_model = joblib.load(f'{DATA_PATH}73번실험/11seeds_ensemble[{i}].pkl')
    models.append(loaded_model)

  0%|          | 0/165 [00:00<?, ?it/s]

In [ ]:
pred_list = []
for model in tqdm(models):
    pred = model.predict(test_ft)
    pred_list.append(pred)

pred = np.mean(pred_list,axis=0)

  0%|          | 0/165 [00:00<?, ?it/s]

In [ ]:
pred.shape

(16800,)

## Submission

In [ ]:
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
submission

,num_date_time,answer
0,1_20220825 00,0
1,1_20220825 01,0
2,1_20220825 02,0
3,1_20220825 03,0
4,1_20220825 04,0
...,...,...
16795,100_20220831 19,0
16796,100_20220831 20,0
16797,100_20220831 21,0
16798,100_20220831 22,0


In [ ]:
submission['answer'] = pred
submission

,num_date_time,answer
0,1_20220825 00,1965.792094
1,1_20220825 01,1890.195339
2,1_20220825 02,1733.434479
3,1_20220825 03,1675.922944
4,1_20220825 04,1655.789332
...,...,...
16795,100_20220831 19,960.388648
16796,100_20220831 20,815.676953
16797,100_20220831 21,741.972034
16798,100_20220831 22,638.939541


In [ ]:
submission.to_csv(f'{DATA_PATH}73번.csv', index=False)

In [ ]:
len(models)

165

In [ ]:
train_ft

,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,기온_3일_이동평균,습도_3일_이동평균,기온_4일_이동평균,습도_4일_이동평균,기온_5일_이동평균,습도_5일_이동평균,기온_6일_이동평균,습도_6일_이동평균,기온_7일_이동평균,습도_7일_이동평균
0,1,18.6,42.0,0,2,6,22,110634.00,39570.00,0.0,...,18.600000,42.000000,18.600000,42.000000,18.600000,42.000000,18.600000,42.000000,18.600000,42.000000
1,1,18.0,45.0,1,2,6,22,110634.00,39570.00,0.0,...,18.300000,43.500000,18.300000,43.500000,18.300000,43.500000,18.300000,43.500000,18.300000,43.500000
2,1,17.7,45.0,2,2,6,22,110634.00,39570.00,0.0,...,18.100000,44.000000,18.100000,44.000000,18.100000,44.000000,18.100000,44.000000,18.100000,44.000000
3,1,16.7,48.0,3,2,6,22,110634.00,39570.00,0.0,...,17.750000,45.000000,17.750000,45.000000,17.750000,45.000000,17.750000,45.000000,17.750000,45.000000
4,1,18.4,43.0,4,2,6,22,110634.00,39570.00,0.0,...,17.880000,44.600000,17.880000,44.600000,17.880000,44.600000,17.880000,44.600000,17.880000,44.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,86.0,19,2,8,34,57497.84,40035.23,0.0,...,24.790278,88.236111,25.161458,87.552083,25.686667,87.275000,25.519444,87.631944,25.341667,86.809524
203996,100,22.4,86.0,20,2,8,34,57497.84,40035.23,0.0,...,24.733333,88.250000,25.127083,87.531250,25.642500,87.266667,25.509028,87.638889,25.327976,86.875000
203997,100,21.3,92.0,21,2,8,34,57497.84,40035.23,0.0,...,24.676389,88.277778,25.094792,87.489583,25.589167,87.308333,25.495833,87.659722,25.321429,86.892857
203998,100,21.0,94.0,22,2,8,34,57497.84,40035.23,0.0,...,24.622222,88.277778,25.063542,87.458333,25.533333,87.375000,25.484028,87.666667,25.311905,86.928571


In [ ]:
test_ft

,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,기온_3일_이동평균,습도_3일_이동평균,기온_4일_이동평균,습도_4일_이동평균,기온_5일_이동평균,습도_5일_이동평균,기온_6일_이동평균,습도_6일_이동평균,기온_7일_이동평균,습도_7일_이동평균
0,1,23.5,72.0,0,3,8,34,110634.00,39570.00,0.0,...,25.693056,72.875000,26.292708,73.385417,26.673333,74.791667,26.521528,76.611111,26.369643,76.553571
1,1,23.0,72.0,1,3,8,34,110634.00,39570.00,0.0,...,25.640278,72.722222,26.253125,73.270833,26.643333,74.600000,26.515278,76.541667,26.367262,76.494048
2,1,22.7,75.0,2,3,8,34,110634.00,39570.00,0.0,...,25.590278,72.597222,26.211458,73.166667,26.613333,74.433333,26.507639,76.493056,26.364881,76.440476
3,1,22.1,78.0,3,3,8,34,110634.00,39570.00,0.0,...,25.537500,72.555556,26.167708,73.093750,26.579167,74.291667,26.496528,76.451389,26.358333,76.404762
4,1,21.8,77.0,4,3,8,34,110634.00,39570.00,0.0,...,25.483333,72.472222,26.129167,73.041667,26.544167,74.141667,26.486111,76.395833,26.352976,76.345238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,22.5,84.0,19,2,8,35,57497.84,40035.23,0.0,...,20.029167,90.416667,20.070833,86.343750,20.470833,83.866667,20.613194,83.451389,20.854762,83.607143
16796,100,20.7,95.0,20,2,8,35,57497.84,40035.23,0.0,...,20.012500,90.708333,20.080208,86.593750,20.455833,83.941667,20.625000,83.527778,20.844643,83.660714
16797,100,20.2,98.0,21,2,8,35,57497.84,40035.23,0.0,...,20.011111,90.902778,20.108333,86.760417,20.447500,83.983333,20.634722,83.618056,20.838095,83.696429
16798,100,20.1,97.0,22,2,8,35,57497.84,40035.23,0.0,...,20.004167,91.097222,20.141667,86.875000,20.442500,83.991667,20.642361,83.666667,20.832738,83.714286
